# Academic Text Simplification Model

This notebook contains a model designed to simplify academic text.

<a href="https://colab.research.google.com/github/ArmandDS/lexical_simplification_bert/blob/master/Lexical_Simplification_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text Simplification

###Importing Libraries

In [ ]:
# Import necessary libraries and modules
!pip install textstat
!pip install tqdm
import numpy as np
from IPython.display import Image
# Get the interactive Tools for Matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from nltk import ngrams
import pandas as pd
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, RepeatVector,Flatten, TimeDistributed, Input,Bidirectional,LocallyConnected1D,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.layers import Embedding, LSTM ,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
# from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
# import tensorflow.keras.utils.to_categorical as to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import math
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import classification_report
import os
import textstat
import nltk
from tqdm import tqdm
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

###Loading Dataset

In [ ]:
# Import necessary libraries and modules
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive"
X_Test = pd.read_csv(path + "/X_Test.txt", sep='\n')
X_Train = pd.read_csv(path + "/X_Train.txt", sep='\n')
Y_Test = pd.read_csv(path + "/Y_Test.txt", sep='\n')
Y_Train = pd.read_csv(path + "/Y_Train.txt", sep='\n')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Perform operations
X_Test = list(X_Test["X_Test"])
X_Train = list(X_Train["X_Train"])
Y_Test = list(Y_Test["Y_Test"])
Y_Train = list(Y_Train["Y_Train"])

In [ ]:
# Perform operations
X_Train = X_Train[0:20000]

###Pre-processing

In [ ]:
# Perform operations
def preprocess(X):
  for i in range(len(X)):
    X[i] = X[i].replace("'",'')
    X[i] = X[i].replace(".",'')

    X[i] = X[i]
  return X
X_Train = preprocess(X_Train)

In [ ]:
# Perform operations
def getVocabulary(data_X):
    #list of sentences
    joined_data_X = " ".join(data_X)
    vocab_list = list(set(joined_data_X.split(" ")))
    return vocab_list

In [ ]:
# Perform operations
Train_Vocab = getVocabulary(X_Train)
Print('Length of Vocab from Train Data:',len(Train_Vocab))

Length of Vocab from Train Data: 45962


###Complexity Identification

In [ ]:
# Perform operations
def mapComplexity(words):
  Complexity_Score = {}
  for word in tqdm(words):
    if textstat.difficult_words(word) == 1:
      Complexity_Score[word]=1
    else:
      Complexity_Score[word]=0
  return Complexity_Score

In [ ]:
# Perform operations
Train_Complexity_Score = mapComplexity(Train_Vocab)

100%|██████████| 45962/45962 [01:02<00:00, 729.65it/s]


In [ ]:
# Perform operations
!pip install transformers
!pip install torch

###Building the BIRT Model

In [ ]:
# Import necessary libraries and modules
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

bert_model = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForMaskedLM.from_pretrained(bert_model)
model.eval()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

To compute the **Zipf values**, we use the library **wordfreq**

In [ ]:
# Perform operations
!pip install wordfreq

In [ ]:
# Perform operations
def generate_candidates(sentence, complex_index):
  candidates = []
  input_list = sentence.split()
  for i in range(len(input_list)):
    tag = pos_tag([input_list[i]])[0][1]
    zipfscore = (zipf_frequency(input_list[i], 'en'))
    replace = False
    if complex_index[i] > 0 and tag in  ['NNS', 'NN', 'VBP', 'RB', 'VBG','VBD' ]:
      replace = True
    if zipfscore < 3.1:
      replace = True
    if replace:
      tagged = '[CLS]'+ sentence.replace(input_list[i], '[MASK]') + ' [SEP] ' + sentence + ' [SEP] '
      counter = 0
      m = 0
      for tok in tokenizer.tokenize(tagged):
        if tok == '[MASK]':
          m = counter
          counter += 1
        counter += 1
      with torch.no_grad():
          network = model(torch.tensor([tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tagged))]), token_type_ids=torch.tensor([[0 for i in range(len(tokenizer.tokenize(tagged)))]]))
          results = network[0][0][m]
      tokens = tokenizer.convert_ids_to_tokens(list(torch.argsort(results, descending=True)[:10]))
      candidates.append((input_list[i],tokens))
  return candidates

In [ ]:
# Import necessary libraries and modules
!pip install textstat
import textstat

In [ ]:
# Perform operations
X_Train = X_Train[0:100]
Y_Train = Y_Train[0:100]

simplified = []

for unsimplified_text in tqdm(X_Train):
  simplified_text=unsimplified_text
  complexity=[textstat.difficult_words(word) for word in unsimplified_text.split()]
  candidates_list=get_bert_candidates(unsimplified_text,complexity)
  for diff_word, l_candidates in candidates_list:
    candidates_order = []
    for w in l_candidates:
      if w.isalpha():
        candidates_order.append((w, zipf_frequency(w, 'en')))
    candidates_order = sorted(candidates_order, key = lambda x: x[1], reverse=True)
    if len(candidates_order) != 0:
      simplified_text = re.sub(diff_word,candidates_order[0][0],simplified_text)
  simplified.append(simplified_text)


  1%|          | 1/100 [00:01<02:57,  1.79s/it]

[('preseason', ['preseason', 'practice', 'playoff', 'reserve', 'training', 'rookie', 'preliminary', 'draft', 'league', 'september']), (',', [',', 'kurt', 'jack', 'and', 'pop', ';', 'john', '.', 'tom', ':']), ('tentative', ["'", 'temporary', 'interim', 'tentative', '′', 'second', 'starting', 'permanent', 'reserve', 'first'])]



  2%|▏         | 2/100 [00:03<02:40,  1.64s/it]

[('twentieth', ['twentieth', 'seventh', 'sixth', 'sixteenth', 'second', 'eighth', 'fifteenth', 'seventeenth', 'tenth', 'eighteenth']), ('Entertainment', ['.', 'entertainment', ';', '!', 'corporation', 'authority', 'championship', 'championships', 'federation', 'company'])]



  3%|▎         | 3/100 [00:04<02:41,  1.66s/it]

...

[('Leonese', ['spanish', 'galician', 'catalan', 'basque', 'portuguese', 'french', 'romani', 'english', 'italian', 'arabic']), ('writer', ['writer', 'poet', 'author', 'painter', 'novelist', 'teacher', 'linguist', 'singer', 'journalist', 'politician']), (',', [',', 'who', ';', '.', ':', '-', 'and', 'he', '...', 'that']), ('Palacios', ['arenas', 'torre', 'santiago', 'castro', 'campos', 'medina', 'vega', 'villa', 'valle', 'alba']), ('Sil', ['sol', 'rio', 'mar', 'sur', 'leon', 'sal', 'valle', 'norte', 'rey', 'real']), ('León', ['leon', 'salamanca', 'madrid', 'monterrey', 'mora', 'spain', 'avila', 'nicaragua', 'malaga', 'zaragoza'])]



 99%|█████████▉| 99/100 [11:25<00:05,  5.14s/it]

[('Grand-Rullecourt', ['ham', 'pont', 'fontaine', 'croix', 'bois', 'faux', 'bellevue', 'nes', 'champs', '##court']), ('commune', ['commune', 'communes', 'village', 'municipality', 'department', 'canton', 'town', 'hamlet', 'community', 'settlement']), ('department', ['department', 'district', 'departments', 'arrondissement', 'prefecture', 'et', 'and', 'dept', 'est', ',']), ('Nord-Pas-de-Calais', ['normandy', 'nord', 'northern', 'northwest', 'north', 'northeast', 'brittany', 'lille', 'northwestern', 'calais']), ('region', ['region', 'area', 'regional', 'district', 'department', 'zone', 'regions', 'province', 'section', 'part'])]



100%|██████████| 100/100 [11:27<00:00,  6.87s/it]

[('reunification', ['reunification', 'unification', '1990', '1945', 'independence', '1994', '1991', '1989', 'partition', 'this']), (',', [',', ':', '-', 'of', '.', 'and', ';', '...', 'when', 'the']), ('Linden', ['linden', 'berlin', 'vienna', 'essen', 'kiel', 'brussels', 'dresden', 'royal', 'baltic', 'dusseldorf']), ('rejoined', ['rejoined', 'joined', 'rejoin', 'left', 'reopened', 'joins', 'abandoned', 'revived', 'reunited', 'returned']), ('operatic', ['operatic', 'opera', 'musical', 'artistic', 'singing', 'theatrical', 'orchestral', 'dramatic', 'concert', 'symphonic'])]


In [ ]:
# Perform operations
data = {'Wiki Unsimplified Text': X_Train, 'Simplified Text': simplified}
df = pd.DataFrame(data)

###Simplification Results

In [ ]:
# Perform operations
df

,Wiki Unsimplified Text,Simplified Text
0,"When Green tore his ACL in a preseason game , Warner took over as the Rams tentative starter",When Green tore his ACL in a training game and Warner took over as the Rams first starter
1,Royal Rumble was the twentieth annual Royal Rumble professional wrestling pay-per-view event produced by World Wrestling Entertainment,Royal Rumble was the second annual Royal Rumble professional wrestling pay-per-view event produced by World Wrestling company
2,Janko Prunk is a Slovenian historian of modern history,peter pop is a polish author of modern history
3,"Sometimes the balalaika is tuned `` guitar style to G-B-D , making it easier to play for Russian guitar players , although balalaika purists frown on this tuning",Sometimes the the is tuned the on style to G-B-D but making it easier to play for Russian on playing but although the players frown on this tuning
4,"Two days after he was sworn in as Prime Minister , the Wall Street Crash of 1929 occurred , marking the beginning of the Great Depression and subsequent Great Depression in Australia",Two days after he was sworn in as Prime Minister during the Wall Street Crash of 1929 happened during mark the beginning of the Great the and the Great the in Australia
...,...,...
95,Arthur is the first reported tropical storm to form in May since Tropical Storm Arlene in 1981,Arthur is the first said tropical storm to form in May since Tropical Storm bob in 1981
96,"She did not perform so well at the World alround championships in Heerenveen , where she finished fifth , but she managed to win the 2006 - 07 Speed Skating World Cup on the 3,000 and 5,000 m track",She did not do so well at the World road games in paris free where she finished fifth free but she managed to win the 2006 and 07 Speed racing World Cup on the 3free000 and 5free000 m track
97,"Leonese language writer , was born in Palacios del Sil in 1918 and died in León in 2007",english language author and was born in villa del real in 1918 and died in spain in 2007
98,Grand-Rullecourt is a commune in the Pas-de-Calais department in the Nord-Pas-de-Calais region of France,ham is a community in the Pas-de-Calais and in the north part of France


In [ ]:
# Perform operations
